# Capacitated EOQ Model

In [ ]:
# Load in the libraries that we'll use

import pandas as pd
import numpy as np

## Model Inputs

Feel free to modify these to understand how the model works. Then `Run All`. 

In [ ]:
#This cell is for reading the system-wide inputs

total_hours = 8736  # this is in years
avail_regular_hours = 4264  # this is in years
avail_time_constraint = avail_regular_hours / total_hours  #expressed as a percentage
nominal_set_up_time = 10  #later we'll change this to a calculation
inv_holding_cost_percent = .2
cost_of_set_up_hour = 100


In [ ]:
#This cell is for reading the product-specific inputs

product_input_table = pd.read_csv("product_input_v1.csv", index_col="Product ID")
product_input_table # display the table

In [ ]:
product_input_table["product_set_up_time_hrs"] = product_input_table["Set Up Scaler-- (q-j)"] * nominal_set_up_time
product_input_table["unit_set_up_cost"] = product_input_table["product_set_up_time_hrs"] * cost_of_set_up_hour
product_input_table

## Internal calculations

In [ ]:
# Rename the input table so that formulas are shorter and more readable

inputs_df = product_input_table.copy()
inputs_df.columns = ["demand", "cost", "rate", "scalar", "set_up_time", "set_up_cost"]

inputs_df

In [ ]:
#create nasty calculation table
#create system-wide paramters  they will be mixed and matched depending on what needs to happen

cogs = (inputs_df["demand"] * inputs_df["cost"]).sum()

sys_S_nomimal_set_up_years = nominal_set_up_time / total_hours
sys_c_S_cost_of_set_up_years = cost_of_set_up_hour / total_hours * total_hours * total_hours #this was in my spreadsheet like this. need to figure out why


internal_calc_df = pd.DataFrame()  
internal_calc_df["Prod_rate_yr_r_j"] = inputs_df["rate"] * total_hours
internal_calc_df["m_over_r"] = inputs_df["demand"] / internal_calc_df["Prod_rate_yr_r_j"]
internal_calc_df["sqrt_2mqic_j"] = np.sqrt(2 * inputs_df["demand"] * inputs_df["scalar"] * inv_holding_cost_percent *inputs_df["cost"] )

c_sum = internal_calc_df["sqrt_2mqic_j"].sum()
alpha = internal_calc_df["m_over_r"].sum()
if alpha > avail_time_constraint:  #this means the problem is infeasible-- production time takes longer than available time
    infeasible_flag = True 
else:
    infeasible_flag = False
    
lambda_var = (c_sum **2 * sys_S_nomimal_set_up_years) / (4*(avail_time_constraint - alpha)**2) - sys_c_S_cost_of_set_up_years 

x = sys_c_S_cost_of_set_up_years
y = sys_S_nomimal_set_up_years
internal_calc_df["Q-1 (S)"] = np.sqrt((2*inputs_df["demand"] * inputs_df["scalar"] * x * y)/(inv_holding_cost_percent * inputs_df["cost"]))
internal_calc_df["Q-2"] = ((c_sum * y) / (avail_time_constraint - alpha)) * np.sqrt((inputs_df["demand"] * inputs_df["scalar"]) / (2 * inv_holding_cost_percent * inputs_df["cost"])) 

print("sys_S_nomimal_set_up_years",sys_S_nomimal_set_up_years)
print("sys_c_S_cost_of_set_up_years",sys_c_S_cost_of_set_up_years)
print("alpha",alpha)
print("cogs",cogs)
print("lambda_var",lambda_var)
internal_calc_df.astype("float").round(3)


## Functions

In [ ]:
# All the functions we plan on using

## Model

In [ ]:
# Optimal Results of capaciated model

optimal_df = pd.DataFrame() 

if lambda_var <= 0:
    optimal_df["q_star_order_size"] = internal_calc_df["Q-1 (S)"]
else:
    optimal_df["q_star_order_size"] = internal_calc_df["Q-2"]

optimal_df["opt_setups_per_year"] = inputs_df["demand"] / optimal_df["q_star_order_size"]
optimal_df["opt_time_in_setups_per_year"] = inputs_df["demand"] * inputs_df["scalar"] * sys_S_nomimal_set_up_years / optimal_df["q_star_order_size"] 

opt_total_setup_percent = optimal_df["opt_time_in_setups_per_year"].sum()

print("opt_total_setup_percent",opt_total_setup_percent)
optimal_df.astype("float").round(4)

In [ ]:
# EOQ, Unconstrained Results -- theorhetical best, but not feasible

eoq_df = pd.DataFrame() 

eoq_df["q_star_eoq"] = np.sqrt((2 * inputs_df["demand"] * inputs_df["scalar"] * y * x) / (inv_holding_cost_percent * inputs_df["cost"] ))

eoq_df["eoq_setups_per_year"] = inputs_df["demand"] / eoq_df["q_star_eoq"]
eoq_df["eoq_time_in_setups_per_year"] = inputs_df["demand"] * inputs_df["scalar"] * sys_S_nomimal_set_up_years / eoq_df["q_star_eoq"] 

eoq_total_setup_percent = eoq_df["eoq_time_in_setups_per_year"].sum()

print("eoq_total_setup_percent",eoq_total_setup_percent)
eoq_df.astype("float").round(4)

In [ ]:
# This is filling out the cost table, both EOQ and Optimal

cost_df = pd.DataFrame() 

#for optimal
cost_df["opt setup cost"] = (inputs_df["demand"] * inputs_df["scalar"] * y * x) / optimal_df["q_star_order_size"]
cost_df["opt hold cost"] = optimal_df["q_star_order_size"] * inv_holding_cost_percent * inputs_df["cost"] / 2

#for EOQ
cost_df["eoq setup cost"] = (inputs_df["demand"] * inputs_df["scalar"] * y * x) / eoq_df["q_star_eoq"]
cost_df["eoq hold cost"] = eoq_df["q_star_eoq"] * inv_holding_cost_percent * inputs_df["cost"] / 2

cost_df.astype("float").round(0)

## Results

In [ ]:
# And here are the results!

print("----- Summary Stats ----------")
print("")
print("------Optimal Plan -----------")

total_holding_inventory_cost = round(cost_df['opt hold cost'].sum())
total_setup_cost = round(cost_df['opt setup cost'].sum())
total_cost = round(total_holding_inventory_cost + total_setup_cost)
avg_working_cap = round(total_holding_inventory_cost / inv_holding_cost_percent)
inv_turns = cogs / avg_working_cap
hours_used = total_hours * (optimal_df["opt_time_in_setups_per_year"].sum() + alpha)
if hours_used >= avail_regular_hours:
    overtime = 0
else:
    overtime = hours_used - avail_regular_hours


print(f"Total Holding Inventory Cost: ${total_holding_inventory_cost:>10,.0f}")
print(f"Total Setup Cost:             ${total_setup_cost:>10,.0f}")
print(f"Total Cost:                   ${total_cost:>10,.0f}")
print("")
print(f"Average Working Capital       ${avg_working_cap:>10,.0f}")
print(f"Inventory Turns               {inv_turns:>10,.1f}")
print("")
print(f"Total Hours- all time          {total_hours:>10,.0f}")
print(f"Total Working Hours            {avail_regular_hours:>10,.0f}")
print(f"Working Hours Used             {hours_used:>10,.0f}")
print(f"Overtime Needed                {overtime:>10,.0f}")

print("")
print("------EOQ Plan (Theoretical Best) -----------")

total_holding_inventory_cost = round(cost_df['eoq hold cost'].sum())
total_setup_cost = round(cost_df['eoq setup cost'].sum())
total_cost = round(total_holding_inventory_cost + total_setup_cost)
avg_working_cap = round(total_holding_inventory_cost / inv_holding_cost_percent)
inv_turns = cogs / avg_working_cap
hours_used = total_hours * (eoq_df["eoq_time_in_setups_per_year"].sum() + alpha)
if hours_used <= avail_regular_hours:
    overtime = 0
else:
    overtime = hours_used - avail_regular_hours


print(f"Total Holding Inventory Cost: ${total_holding_inventory_cost:>10,.0f}")
print(f"Total Setup Cost:             ${total_setup_cost:>10,.0f}")
print(f"Total Cost:                   ${total_cost:>10,.0f}")
print("")
print(f"Average Working Capital       ${avg_working_cap:>10,.0f}")
print(f"Inventory Turns               {inv_turns:>10,.1f}")
print("")
print(f"Total Hours- all time          {total_hours:>10,.0f}")
print(f"Total Working Hours            {avail_regular_hours:>10,.0f}")
print(f"Working Hours Used             {hours_used:>10,.0f}")
print(f"Overtime Needed                {overtime:>10,.0f}")




## Round to nearest power of 2

Take the optimal solution, round the number of set ups per year to the nearest power of two, then recalculate the solution.

In [ ]:
# Remember what the table looks like
optimal_df.round(4)

In [ ]:
# round to nearest power of 2

def round_to_power_of_2(x):
    """Round x to the nearest power of 2. Returns the power of 2.
    For example, 3.2 rounds to 4.
    Break ties going up."""

    if np.isnan(x):
        # Non-number input
        return np.nan
    elif np.log2(x) % 1 == 0:
        # Already a power of 2
        return np.log2(x)
    else:
        # Round
        lower = 2 ** np.floor(np.log2(x))
        upper = 2 ** np.ceil(np.log2(x))

        if upper - x <= x - lower:
            return upper
        else:
            return lower
        


In [ ]:
# Test the function

round_to_power_of_2(50)

In [ ]:
# Start building the rounded table 
rounded_df = pd.DataFrame()

rounded_df["rounded_setups_per_year"] = optimal_df["opt_setups_per_year"].apply(round_to_power_of_2)
rounded_df # not a very interesting example dataset!